# Ingest ADS Notebook Files

![files_to_database](files_to_database.png)

This Notebook will load selected files (from a local or networked file system) into a SQL Server database.

> This is a stand-alone Notebook 'Project', and does use an associated eltSnap project. For best results, verify/update the **Notebook Configuration** code cell below, and then use the 'Run all' option for this Notebook.

## Notebook Configuration

In [1]:
# Configuration Parameters
$FolderLocation = 'C:\ADS\Repository\'
$ServerInstance = 'SRV6'
$Database = 'eltsnap_v2'

# Set Begin Time
$BegTime = $(Get-Date)
'Beg Time: ' +  $BegTime

# Show Environment
'Run Values:'
'Folder Location: ' + $FolderLocation
'Server: ' + $ServerInstance
'Database: ' +$Database

## Reset staging table

In [2]:
Invoke-Sqlcmd -ServerInstance $ServerInstance -Database $Database -Query "TRUNCATE TABLE [stg].[files]"

Beg Time: 01/07/2021 08:23:54
Run Values:
Folder Location: C:\ADS\Repository\
Server: SRV6
Database: eltsnap_v2
. {
>> Invoke-Sqlcmd -ServerInstance $ServerInstance -Database $Database -Query "TRUNCATE TABLE [stg].[files]"
>> }
>> 


## Load Files into table: [stg].[files]


In [3]:
Get-ChildItem -Path $FolderLocation -Recurse -Include *.ipynb | Select-Object @{l="FullName";e={($_.FullName).Replace($FolderLocation,"")}}, BaseName, @{l="Extension";e={($_.Extension).Replace(".","")}}, LastWriteTime, Length, @{l="Contents";e={(Get-Content $_ -Raw).ToString()}} | Write-SqlTableData -ServerInstance $ServerInstance -DatabaseName $database -SchemaName "stg" -TableName "files"

## Merge table: [stg].[files] into: [ads].[files]

In [4]:
Invoke-Sqlcmd -ServerInstance $ServerInstance -Database $Database -Query "EXEC [ads].[Merge Staged Files];"


row_count
---------
        3




## Ingest all Notebooks Files into table: [ads].[notebooks]

In [5]:
Invoke-Sqlcmd -ServerInstance $ServerInstance -Database $Database -Query "EXEC [ads].[Ingest All Notebooks];"

## Run Summary

In [6]:
'Beg Time: ' +  $BegTime

$EndTime = $(Get-Date)
'End Time: ' +  $EndTime

$EndTime - $BegTime


Beg Time: 01/07/2021 08:23:54
End Time: 01/07/2021 08:24:13


Days              : 0
Hours             : 0
Minutes           : 0
Seconds           : 19
Milliseconds      : 527
Ticks             : 195273973
TotalDays         : 0.000226011542824074
TotalHours        : 0.00542427702777778
TotalMinutes      : 0.325456621666667
TotalSeconds      : 19.5273973
TotalMilliseconds : 19527.3973



